## Community detection for all metropolitan areas

In [122]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import networkx as nx

import scipy
import csv

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import json
import community as community_louvain
from copy import deepcopy
# from modularity_maximization.utils import get_modularity

from itertools import product

import networkx.algorithms.community as nx_comm

from scipy.spatial.distance import pdist, squareform

%matplotlib inline

In [123]:
# ez tolti be az octave cell magicet
#    %load_ext oct2py.ipython

In [124]:
from oct2py import octave
#octave.addpath('/home/ubuntu/GenLouvain/')
#octave.addpath('/home/ubuntu/GenLouvain/private/')
octave.addpath('/home/barcsab/urban_communities/scripts')
octave.addpath('/home/ubuntu/GenLouvain/')
octave.addpath('/home/ubuntu/GenLouvain/private/')

'.:/home/ubuntu/GenLouvain:/home/barcsab/urban_communities/scripts:/home/ubuntu/GenLouvain/private:/home/ubuntu/anaconda3/lib/python3.7/site-packages/oct2py:/home/ubuntu/anaconda3/lib/python3.7/site-packages/octave_kernel:/usr/lib/x86_64-linux-gnu/octave/4.2.2/site/oct/x86_64-pc-linux-gnu:/usr/lib/x86_64-linux-gnu/octave/site/oct/api-v51/x86_64-pc-linux-gnu:/usr/lib/x86_64-linux-gnu/octave/site/oct/x86_64-pc-linux-gnu:/usr/share/octave/4.2.2/site/m:/usr/share/octave/site/api-v51/m:/usr/share/octave/site/m:/usr/share/octave/site/m/startup:/usr/lib/x86_64-linux-gnu/octave/4.2.2/oct/x86_64-pc-linux-gnu:/usr/share/octave/4.2.2/m:/usr/share/octave/4.2.2/m/audio:/usr/share/octave/4.2.2/m/debian:/usr/share/octave/4.2.2/m/deprecated:/usr/share/octave/4.2.2/m/elfun:/usr/share/octave/4.2.2/m/general:/usr/share/octave/4.2.2/m/geometry:/usr/share/octave/4.2.2/m/gui:/usr/share/octave/4.2.2/m/help:/usr/share/octave/4.2.2/m/image:/usr/share/octave/4.2.2/m/io:/usr/share/octave/4.2.2/m/java:/usr/share/

In [125]:
# !pwd

### data

In [126]:
# data IN
mobility = pd.read_csv("../data/usageousers_city_mobility_CT_networks.rpt.gz") ## A MOBILITY HÁLÓBÓL SZÁMOLOK POSITION-T, AZÉRT KELL!

follow_hh = pd.read_csv("../data/usageousers_city_follower_CT_HH_networks.rpt.gz")
follow_hh = follow_hh.rename(columns={"tract_home.1": "tract_home_1"})

cbsacode = pd.read_csv("../data/cbsacode_shortname_tracts.csv",sep=";", index_col=0)

# census data
census = pd.read_csv("../data/censusdata_top50_2012.csv")
census_2 = pd.read_csv("../data/censusdata_top50_2017.csv")

# reading geojson data, converting it to geopandas dataframe
tract_geoms = gpd.GeoDataFrame.from_features(
    [json.loads(e.strip('\n')) for e in open('../data/censustract_geoms_top50.geojson').readlines()]
)

In [127]:
tract_geoms['centroid'] = tract_geoms['geometry'].centroid

In [128]:
# derékszögű koordináták
tract_center_dict = tract_geoms\
    .set_geometry('centroid',crs={'init':'epsg:4326'})\
    .to_crs({'init':'epsg:3785'})\
    .set_index('full_geoid')['centroid'].map(lambda p: p.coords[0]).to_dict()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [129]:
# choose a city
##city_code = cbsacode[cbsacode.short_name == city].iloc[0].cbsacode

In [130]:

city_l = cbsacode.short_name.unique()

In [131]:
city_code_l = cbsacode.cbsacode.unique()

# RÉGI
# filter
mobility = mobility[mobility["cbsacode"] == city_code]
follow_hh = follow_hh[follow_hh["cbsacode"] == city_code]

# create graphs
# create empty graphs
g_mob = nx.Graph() # mobility graph - weights are counts
## g_mob_lw = nx.Graph() # log edge weights - new_weight=log(1+weight) 
## g_mob_Nw = nx.Graph() # with NO WEIGHTS

g_fol_hh = nx.Graph() # follow home-home graph - weights are counts
## g_fol_hh_lw = nx.Graph() # graphs with log edge weights - new_weight=log(1+weight) 
## g_fol_hh_Nw = nx.Graph() # graphs with NO WEIGHTS


# fill in the networks with data
mobility['w_edges'] = list(zip(mobility.tract_home,mobility.tract_work,mobility.cnt))
g_mob.add_weighted_edges_from(mobility["w_edges"], weight='cnt')

follow_hh['w_edges'] = list(zip(follow_hh.tract_home,follow_hh.tract_home_1,follow_hh.cnt))
g_fol_hh.add_weighted_edges_from(follow_hh["w_edges"], weight='cnt')


## graph with log weights
## mobility['cnt_2']=np.log(1+mobility['cnt'])
## mobility['w_edges_2']=list(zip(mobility.tract_home,mobility.tract_work,mobility.cnt_2))
## g_mob_lw.add_weighted_edges_from(mobility["w_edges_2"], weight='cnt_2')

## follow_hh['cnt_2']=np.log(1+follow_hh['cnt'])
## follow_hh['w_edges_2']=list(zip(follow_hh.tract_home,follow_hh.tract_home_1,follow_hh.cnt_2))
## g_fol_hh_lw.add_weighted_edges_from(follow_hh["w_edges_2"], weight='cnt_2')


## graph with NO WEIGHTS 
## mobility['edges'] = list(zip(mobility.tract_home,mobility.tract_work))
## g_mob_Nw.add_edges_from(mobility['edges'])

## follow_hh['edges'] = list(zip(follow_hh.tract_home,follow_hh.tract_home_1))
## g_fol_hh_Nw.add_edges_from(follow_hh['edges'])


## KERDES SELF LOOPS???? - EZ NEM BIZTOS, HOGY MŰKÖDIK!!!!!!
# remove self loops
g_mob.remove_edges_from(nx.selfloop_edges(g_mob))
## g_mob_lw.remove_edges_from(nx.selfloop_edges(g_mob_lw))
## g_mob_Nw.remove_edges_from(nx.selfloop_edges(g_mob_Nw))

g_fol_hh.remove_edges_from(nx.selfloop_edges(g_fol_hh))
## g_fol_hh_lw.remove_edges_from(nx.selfloop_edges(g_fol_hh_lw))
## g_fol_hh_Nw.remove_edges_from(nx.selfloop_edges(g_fol_hh_Nw))

In [132]:
# choose a graph
# G = g_fol_hh

# RÉGI

coords = np.array([tract_center_dict[n] for n in G.nodes()])

d = pdist(coords)
D = squareform(pdist(coords))

max_dist = np.amax(D)

# adjacency matrix
A = nx.adjacency_matrix(G)

#### KERDES -  jo, ha az importance mindig a mobilityből van számolva??????
# importance - number of users
tract_outdeg_mob = mobility.groupby('tract_home')[['cnt']].sum()
N = np.matrix([tract_outdeg_mob.loc[k].iloc[0] for k in G.nodes()]).T

# binsize
b = max_dist/99 # number of bins = 100

In [133]:
def create_graphs(city, mobility, follow_hh):
    city_code = cbsacode[cbsacode.short_name == city].iloc[0].cbsacode
    mob_df = mobility[mobility["cbsacode"] == city_code]
    fol_hh_df = follow_hh[follow_hh["cbsacode"] == city_code]

    # create graphs
    # create empty graphs
    g_mob = nx.Graph() # mobility graph - weights are counts
    g_fol_hh = nx.Graph() # follow home-home graph - weights are counts

    # fill in the networks with data
    mob_df['w_edges'] = list(zip(mob_df.tract_home,mob_df.tract_work,mob_df.cnt))
    g_mob.add_weighted_edges_from(mob_df["w_edges"], weight='cnt')

    fol_hh_df['w_edges'] = list(zip(fol_hh_df.tract_home,fol_hh_df.tract_home_1,fol_hh_df.cnt))
    g_fol_hh.add_weighted_edges_from(fol_hh_df["w_edges"], weight='cnt')
    
     ## KERDES SELF LOOPS???? - EZ NEM BIZTOS, HOGY MŰKÖDIK!!!!!!
    # remove self loops
    g_mob.remove_edges_from(nx.selfloop_edges(g_mob))
    g_fol_hh.remove_edges_from(nx.selfloop_edges(g_fol_hh))
    return (g_mob, g_fol_hh)

In [134]:
# CONSENSUS CLUSTERING
def consen(csv,G):    
    iters = pd.read_csv(csv)

    geoid_pairs = list(product(list(iters.geoid), list(iters.geoid)))

    consen_df = pd.DataFrame(geoid_pairs, columns=['geoid_1','geoid_2'])

    # remove selfloops
    consen_df = consen_df[consen_df.geoid_1!=consen_df.geoid_2]

    iters = iters.set_index('geoid')

    iters['clusts'] = iters.values.tolist()

    consen_df = pd.merge(consen_df, iters['clusts'], left_on = 'geoid_1', right_on = 'geoid')
    consen_df = pd.merge(consen_df, iters['clusts'], left_on = 'geoid_2', right_on = 'geoid')

    consen_df = consen_df.rename(columns = {'clusts_x': 'clusts_1', 'clusts_y': 'clusts_2'})

    def same(row):
        return np.array(np.equal(row['clusts_1'], row['clusts_2'])).astype(int).sum()

    # how many times do the two tracts (geoid_1 and geoid_2) clustered to the same community?
    # --> weights of graf on which clustering gives the consensus clustering
    consen_df['w'] = consen_df.apply(lambda row: same(row), axis=1)
    print(consen_df.head())

    # graph for consensus clustering
    g_cons = nx.Graph() 
    consen_df['w_edges'] = list(zip(consen_df.geoid_1,consen_df.geoid_2,consen_df.w))
    g_cons.add_weighted_edges_from(consen_df["w_edges"], weight='w')
    print(g_cons.nodes())

    s_louv = community_louvain.best_partition(g_cons, weight='w')
    print(s_louv)

    S = np.array([np.array([s_louv[n]]) for n in G.nodes()])
    return S

In [135]:
#consen('consensusNew York.csv',g_fol_hh)

In [136]:
cons_clust_df =  cbsacode.set_index('geoid')

In [137]:
len(city_l)

50

In [138]:
g_mob, g_fol_hh = create_graphs('Boston', mobility, follow_hh)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [115]:
consen_df = pd.DataFrame()
consen_df['geoid'] = G.nodes()
S_df = pd.DataFrame()
A = nx.adjacency_matrix(G)
coords = np.array([tract_center_dict[n] for n in G.nodes()]) ## KERDES ez jó így? tract_center_dict
d = pdist(coords)
D = squareform(pdist(coords))
#### KERDES -  jo, ha az importance mindig a mobilityből van számolva??????
# importance - number of users
tract_outdeg_mob = mobility.groupby('tract_home')[['cnt']].sum()
N = np.matrix([tract_outdeg_mob.loc[k].iloc[0] for k in G.nodes()]).T
max_dist = np.amax(D)
b = max_dist/99

In [112]:
city_l

array(['New York', 'Los Angeles', 'Riverside', 'San Diego', 'Chicago',
       'Dallas', 'Philadelphia', 'Houston', 'Washington', 'Miami',
       'Atlanta', 'Boston', 'Providence', 'San Francisco', 'San Jose',
       'Detroit', 'Phoenix', 'Seattle', 'Minneapolis', 'St. Louis',
       'Tampa', 'Baltimore', 'Denver', 'Pittsburgh', 'Portland',
       'Charlotte', 'Sacramento', 'San Antonio', 'Orlando', 'Cincinnati',
       'Cleveland', 'Kansas City', 'Las Vegas', 'Columbus',
       'Indianapolis', 'Austin', 'Virginia Beach', 'Nashville',
       'Milwaukee', 'Jacksonville', 'Memphis', 'Oklahoma City',
       'Louisville/Jefferson County', 'Hartford', 'Richmond',
       'New Orleans', 'Buffalo', 'Raleigh', 'Birmingham',
       'Salt Lake City'], dtype=object)

In [ ]:
for city in city_l:
    g_mob, g_fol_hh = create_graphs(city, mobility, follow_hh)
    for (G, g_type) in [(g_mob, 'mob'), (g_fol_hh, 'fol_hh')]:
        consen_df = pd.DataFrame()
        consen_df['geoid'] = G.nodes()
        S_df = pd.DataFrame()
        A = nx.adjacency_matrix(G)
        coords = np.array([tract_center_dict[n] for n in G.nodes()]) ## KERDES ez jó így? tract_center_dict
        d = pdist(coords)
        D = squareform(pdist(coords))
        #### KERDES -  jo, ha az importance mindig a mobilityből van számolva??????
        # importance - number of users
        tract_outdeg_mob = mobility.groupby('tract_home')[['cnt']].sum()
        N = np.matrix([tract_outdeg_mob.loc[k].iloc[0] for k in G.nodes()]).T
        # binsize
        max_dist = np.amax(D)
        b = max_dist/99 # number of bins = 100      
        for _ in range(3):
            print(_)
            Ms,Mgn,Dfn = octave.ModularitySpaGN(A,D,N,b, nout = 3)
            S,Q,n_it = octave.iterated_genlouvain(Ms, nout=3)
            S_df[len(S_df.columns)] = S.T[0]
        S_df['geoid'] = list(G.nodes()) ## KERDES JO???
        S_df = S_df.set_index('geoid')
        csv_name = 'consensus' + city + '_' + g_type + '.csv'
        S_df.to_csv('../data/'+ csv_name)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


0


In [ ]:
        S_cons = consen(csv,G) ## eddig ment HIBA, KÉRDÉS
        consen_df['S_cons'] = S_cons
        consen_df['city'] = city
        consen_df['type'] = g_type
        consen_df = consen_df.set_index('geoid')
        pd.merge(cons_clust_df, consen_df, left_index = True, right_index = True)
cons_clust_df.to_csv('all_cons.csv')

In [ ]:
csv_city = str(G) + '_rawclust.csv'+ city
        with open(csv_city, mode='w') as clust_file:
            clust_writer = csv.writer(clust_file, delimiter=',', quotechar='"') ### ez kerdeses!!!!

   

    employee_writer.writerow(['John Smith', 'Accounting', 'November'])

In [106]:
tract_sum = cbsacode.groupby('cbsacode')[['cbsacode']].count() # number of tracts per city
tract_sum = tract_sum.rename(columns={'cbsacode': 'sum_tracts'})

In [102]:
# unique values in the network counted by side of edge

mob_u_h = mobility.tract_home.tolist()
mob_u_w = mobility.tract_work.tolist()
u_geoid_mob = set([*mob_u_h,*mob_u_w])
cbsacode['in_mob'] = cbsacode.geoid.isin(u_geoid_mob).astype(int)

fol_u_h = follow_hh.tract_home.tolist()
fol_u_h1 = follow_hh.tract_home_1.tolist()
u_geoid_fol = set([*fol_u_h,*fol_u_h1])
cbsacode['in_fol'] = cbsacode.geoid.isin(u_geoid_fol).astype(int)

In [113]:
netw_count = cbsacode.groupby('cbsacode')[['in_mob','in_fol']].sum()

In [114]:
city_df = pd.merge(tract_sum, netw_count, left_index=True, right_index=True)

In [ ]:
nx_comm.modularity()

G = nx.barbell_graph(3, 0)

nx_comm.modularity(G, [{0, 1, 2}, {3, 4, 5}])
0.35714285714285715

nx_comm.modularity(G, nx_comm.label_propagation_communities(G))
0.35714285714285715


In [115]:
city_df

,sum_tracts,in_mob,in_fol
cbsacode,,,
12060.0,947,947,940
12420.0,348,348,344
12580.0,674,674,666
13820.0,263,263,261
14460.0,990,990,983
15380.0,295,295,292
16740.0,536,536,528
16980.0,2201,2201,2176
17140.0,495,495,492


In [12]:
tract = mobility.groupby('tract_home')[['cnt']].sum()

In [13]:
tract

,cnt
tract_home,
14000US01007010001,9
14000US01007010002,14
14000US01007010003,10
14000US01007010004,22
14000US01009050101,21
14000US01009050102,29
14000US01009050200,20
14000US01009050300,12
14000US01009050400,1


In [ ]:
follow_hh = follow_hh.merge(tract_outdeg_mob, left_on='tract_home', right_index=True)

N = np.matrix([tract_outdeg_mob.loc[k].iloc[0] for k in g_fol_hh.nodes()]).T
# modify N